<a href="https://colab.research.google.com/github/alex-jk/buyers-ID-manual-project/blob/main/process_studies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook extracts and cleans text from PDF reports for use in training the buyer profile extractor model.**

**Add input and output examples to jsonl file**

In [1]:
!git clone https://github.com/alex-jk/buyers-ID-manual-project.git

Cloning into 'buyers-ID-manual-project'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 97 (delta 46), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (97/97), 2.88 MiB | 12.71 MiB/s, done.
Resolving deltas: 100% (46/46), done.


**Install and Import libraries**

In [2]:
!pip uninstall -y nltk
!rm -rf /root/nltk_data /content/nltk_data /usr/local/nltk_data

# ✅ Step 2: Reinstall clean version
!pip install nltk==3.8.1

# Install specific compatible versions for torch 2.3.1 / cu121
!pip install torch==2.3.1+cu121 torchaudio==2.3.1+cu121 torchvision==0.18.1+cu121 -f https://download.pytorch.org/whl/torch_stable.html

# Install the other pinned versions
!pip install transformers==4.41.2 accelerate==0.31.0 --no-deps

# Install compatible tokenizers version LAST to ensure it sticks
!pip install tokenizers==0.19.1

# Ensure base dependencies are present (redundant ok)
!pip install bitsandbytes sentencepiece

print("\nInstalled specific package versions (including compatible tokenizers). PLEASE RESTART RUNTIME NOW.")

Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 57.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textblob 0.19.0 requires nltk>=3.9, but you have nltk 3.8.1 which is incompatible.
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

**After install  - RESTART session then RUN FROM HERE**

In [1]:
%cd buyers-ID-manual-project
!ls

/content/buyers-ID-manual-project
buyers-manual-article-01.txt  process_studies.ipynb
data			      prompt_main_idea.txt
extraction_utils.py	      prompt_template.txt
extract_text_from_pdfs.ipynb  README.md
LICENSE


In [2]:
import pandas as pd
import json
import sys
import os
import traceback

In [3]:
import nltk
print("Attempting NLTK 'punkt' download directly...")
nltk.download('punkt') # Remove quiet=True and try/except
print("NLTK download command finished.")

Attempting NLTK 'punkt' download directly...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


NLTK download command finished.


**Read input output file and write to jsonl**

In [4]:
input_output_df = pd.read_csv("data/buyers_id_manual_input_output.csv")

print(input_output_df.shape)
print(input_output_df.columns)
print("\n-------------------------------\n")
print(input_output_df.head())

(9, 3)
Index(['Study', 'Input', 'Output'], dtype='object')

-------------------------------

                                    Study  \
0  The Shapiro Group Georgia Demand Study   
1  The Shapiro Group Georgia Demand Study   
2  The Shapiro Group Georgia Demand Study   
3  The Shapiro Group Georgia Demand Study   
4  The Shapiro Group Georgia Demand Study   

                                               Input  \
0  Almost half these men are the age 30-39, with ...   
1  The data clearly debunk the myth that CSEC is ...   
2  Not only are 65% of men who buy sex with young...   
3  Craigslist is by far the most efficient medium...   
4  While many of the men who exploit these childr...   

                                              Output  
0  Almost half these men are the age 30-39, with ...  
1  Men who\nrespond to advertisements for sex wit...  
2  65% of men who buy sex with young females do s...  
3  Buyers respond to Craigslist ads 3 times more ...  
4  Nearly half of buyers

In [5]:
input_output_df = input_output_df.drop_duplicates(subset=["Study", "Input", "Output"])

# Convert to JSONL
with open("data/labeled_chunks.jsonl", "w") as f:
    for _, row in input_output_df.iterrows():
        json_obj = {
            "study": row["Study"],
            "input": row["Input"],
            "output": row["Output"]
        }
        f.write(json.dumps(json_obj) + "\n")

**Models for text extraction**

In [32]:
%cd /content/buyers-ID-manual-project

!git pull

/content/buyers-ID-manual-project
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 970 bytes | 970.00 KiB/s, done.
From https://github.com/alex-jk/buyers-ID-manual-project
   2c65560..7be3fbd  main       -> origin/main
Updating 2c65560..7be3fbd
Fast-forward
 extraction_utils.py | 3 ++-
 1 file changed, 2 insertions(+), 1 deletion(-)


In [33]:
import importlib
import extraction_utils

# Reload the entire module
importlib.reload(extraction_utils)
print("Module 'extraction_utils' reloaded.")

from extraction_utils import extract_information, chunk_text_by_tokens, process_text_chunks_with_prompt

print("Functions imported from extraction_utils successfully.")

Module 'extraction_utils' reloaded.
Functions imported from extraction_utils successfully.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [10]:
# Check installed versions of required packages
!pip list | grep -E 'transformers|torch|accelerate|flash-attn'

accelerate                            0.31.0
sentence-transformers                 3.4.1
torch                                 2.3.1+cu121
torchaudio                            2.3.1+cu121
torchsummary                          1.5.1
torchvision                           0.18.1+cu121
transformers                          4.41.2


In [11]:
# --- Configuration ---
model_id = "microsoft/Phi-3-mini-4k-instruct"

# --- Load Model and Tokenizer ---
# Load the model with 4-bit quantization to save memory (requires bitsandbytes)
# Use device_map="auto" to automatically use GPU if available
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",         # Use GPU if available, otherwise CPU
        torch_dtype="auto",        # Automatically select appropriate dtype
        trust_remote_code=True,    # Phi-3 requires this
        # Optional: uncomment below for 4-bit loading (needs bitsandbytes)
        # load_in_4bit=True,
        # bnb_4bit_compute_dtype=torch.bfloat16
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    print(f"Model '{model_id}' loaded successfully.")

    # --- Create a Hugging Face Pipeline for easier text generation ---
    # Note: max_new_tokens controls how long the generated output can be. Adjust as needed.
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        # Adjust max_new_tokens if your extracted text might be longer
        # Needs to be long enough for the longest expected extraction + "NONE"
        max_new_tokens=256,
        # Temperature=0 means more deterministic output, higher means more creative/random
        # temperature=0.0,
        # top_p=0.95, # Optional: nucleus sampling
        do_sample=False # Set to False for more deterministic extraction
    )

except Exception as e:
    print(f"Error loading model or creating pipeline: {e}")
    print("Ensure you have sufficient RAM/VRAM and necessary libraries installed.")
    print("Consider using Google Colab with a T4 GPU runtime.")
    # Exit if model loading fails
    exit()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model 'microsoft/Phi-3-mini-4k-instruct' loaded successfully.


**Example Usage**

In [12]:
input_text_1 = "Almost half these men are the age 30-39, with the next largest group being men under age 30. The mean age is 33 and the median 31. The youngest survey participant was 18, and the oldest was 67."

input_text_2 = "The data clearly debunk the myth that CSEC is a problem relegated to the urban core. Men who respond to advertisements for sex with young females come from all over metro Atlanta, the geographic market where the advertisements in this study were targeted."

input_text_3 = "This paragraph discusses unrelated economic factors in the region and contains no information about buyers or traffickers."

input_text_4 = "Research indicates traffickers often groom potential buyers by displaying luxury goods online and frequenting specific forums known for risky behavior discussions. Victims may appear withdrawn or display signs of coaching. Reporting suspicions anonymously through the national hotline is encouraged."

In [13]:
prompt_buyers_profiles = "prompt_template.txt"

In [14]:
!cat /content/buyers-ID-manual-project/extraction_utils.py

import torch
import os
import re
import nltk
nltk.download('punkt')

# --- Function to Load a Specific Prompt File ---
def load_system_prompt(prompt_filename):
    """Loads system prompt text from a specified file."""
    system_prompt = ""
    try:
        # Assumes prompt file is in the same directory as this script
        script_dir = os.path.dirname(__file__)
        prompt_path = os.path.join(script_dir, prompt_filename)
        with open(prompt_path, 'r', encoding='utf-8') as f:
            system_prompt = f.read()
        if not system_prompt:
            print(f"WARNING: Prompt file loaded but is empty: {prompt_path}")
            return f"ERROR: Prompt file empty {prompt_filename}" # Return specific error
    except FileNotFoundError:
        print(f"ERROR: Prompt file not found at {prompt_path}")
        return f"ERROR: Prompt file not found {prompt_filename}" # Return specific error
    except Exception as e:
        print(f"ERROR: Could not read prompt file {prompt_path}: 

In [15]:
print("\n--- Extracting Information ---")

# Check if the pipeline object exists before trying to use it
if 'pipe' in locals() and pipe is not None:
    print(f"\nRunning prompt: {prompt_buyers_profiles}")

    print(f"\nInput 1:\n{input_text_1}")
    extraction_1 = extract_information(input_text_1, pipe, prompt_buyers_profiles)
    print(f"\nExtraction 1:\n{extraction_1}")

    print("-" * 20)

    print(f"\nInput 2:\n{input_text_2}")
    extraction_2 = extract_information(input_text_2, pipe, prompt_buyers_profiles)
    print(f"\nExtraction 2:\n{extraction_2}")

    print("-" * 20)

    print(f"\nInput 3:\n{input_text_3}")
    extraction_3 = extract_information(input_text_3, pipe, prompt_buyers_profiles)
    print(f"\nExtraction 3:\n{extraction_3}")

    print("-" * 20)

    print(f"\nInput 4:\n{input_text_4}")
    extraction_4 = extract_information(input_text_4, pipe, prompt_buyers_profiles)
    print(f"\nExtraction 4:\n{extraction_4}")

else:
    print("\nERROR: The 'pipe' object (text generation pipeline) was not found or not created successfully.")
    print("Please ensure the model loading cell was run successfully after importing functions.")


--- Extracting Information ---

Running prompt: prompt_template.txt

Input 1:
Almost half these men are the age 30-39, with the next largest group being men under age 30. The mean age is 33 and the median 31. The youngest survey participant was 18, and the oldest was 67.



Extraction 1:
Almost half these men are the age 30-39, with the next largest group being men under age 30. The mean age is 33 and the median 31. The youngest survey participant was 18, and the oldest was 67.
--------------------

Input 2:
The data clearly debunk the myth that CSEC is a problem relegated to the urban core. Men who respond to advertisements for sex with young females come from all over metro Atlanta, the geographic market where the advertisements in this study were targeted.

Extraction 2:
Men who respond to advertisements for sex with young females come from all over metro Atlanta.
--------------------

Input 3:
This paragraph discusses unrelated economic factors in the region and contains no information about buyers or traffickers.

Extraction 3:
NONE
--------------------

Input 4:
Research indicates traffickers often groom potential buyers by displaying luxury goods online and frequenting specific forums known for risky behavior discussions. Victims may appear withdr

**Research Articles - extract main ideas**

In [25]:
# 1. DEFINE INPUT and PROMPT FILENAMES
input_txt_filename = "buyers-manual-article-01.txt"  # <--- CHANGE to your uploaded .txt filename
prompt_to_use = "prompt_main_idea.txt"    # <--- CHANGE to the prompt file you want to apply

input_txt_filepath = os.path.join('/content/buyers-ID-manual-project', input_txt_filename)

In [26]:
!ls /content/

buyers-ID-manual-project  sample_data


In [34]:
# 2. Check if pipeline 'pipe' exists from previous cells
prompts_to_run = [
    "prompt_main_idea.txt"
]

# --- Configuration ---
summaries_output_filepath = "extracted_summaries.txt" # Dedicated file for summaries

CHUNK_MAX_TOKENS = 2500
BATCH_SIZE = 1

print(f"--- Processing Started ---")
print(f"Input file: {input_txt_filepath}")
print(f"Attempting to save summaries to: {summaries_output_filepath}")

# Check if pipeline and tokenizer are ready (print to console)
if 'pipe' in locals() and callable(pipe) and 'tokenizer' in locals() and hasattr(tokenizer, 'encode'):
    print(f"Pipeline and tokenizer are ready.")
    try:
        # --- Read Input File --- (print to console)
        print(f"Reading text file: {input_txt_filepath}")
        with open(input_txt_filepath, 'r', encoding='utf-8') as f:
            full_text_content = f.read()
        print(f"Successfully read {len(full_text_content)} characters.")

        if full_text_content:
            # --- Chunk Text --- (print to console)
            print(f"Chunking text with max_chunk_tokens = {CHUNK_MAX_TOKENS}...")
            text_chunks = chunk_text_by_tokens(full_text_content, tokenizer, max_chunk_tokens=CHUNK_MAX_TOKENS)

            if text_chunks:
                # Optional Debug: Print Chunk Token Counts (print to console)
                # print("\n--- Checking Chunk Token Counts ---")
                # for i, chunk in enumerate(text_chunks):
                #     try:
                #         encoded_output = tokenizer.encode(chunk, add_special_tokens=False)
                #         chunk_tokens = len(encoded_output)
                #         print(f"Chunk {i+1} token count: {chunk_tokens}")
                #     except Exception as e: print(f"Could not tokenize chunk {i+1}: {e}")
                # print("------------------------------------\n")

                # --- Process Chunks & Write Summaries ---
                all_results_by_prompt = {} # Still useful for potential later use or debugging

                # Open the summaries file in 'write' mode ONCE before the loop
                # This ensures the file is cleared for each new run of the script
                try:
                    with open(summaries_output_filepath, 'w', encoding='utf-8') as summary_file:
                        print(f"Opened '{summaries_output_filepath}' for writing summaries.")

                        print(f"\nProcessing prompts: {prompts_to_run}")
                        for prompt_file in prompts_to_run:
                            # Status updates print to console
                            print(f"\n>>> Starting processing for prompt file: {prompt_file} <<<")

                            # Call the batch processing function
                            final_results = process_text_chunks_with_prompt(
                                text_chunks=text_chunks,
                                generation_pipeline=pipe,
                                prompt_filename_to_use=prompt_file,
                                batch_size=BATCH_SIZE, # Use the adjusted batch size
                                max_new_tokens=512     # Keep or adjust as needed
                            )
                            all_results_by_prompt[prompt_file] = final_results # Store if needed

                            # --- Write ONLY the collected summaries to the file ---
                            summary_file.write(f"--- Summaries for Prompt: {prompt_file} ---\n\n")
                            if final_results:
                                print(f"  Writing {len(final_results)} summaries to file...")
                                for i, summary in enumerate(final_results):
                                    summary_file.write(f"Summary {i+1}:\n")
                                    summary_file.write(summary)
                                    summary_file.write("\n--------------------\n") # Separator
                                summary_file.write("\n") # Add space before next prompt section
                            else:
                                print(f"  No relevant summaries extracted for '{prompt_file}'. Writing note to file.")
                                summary_file.write(f"No relevant information extracted using this prompt.\n\n")
                            # --------------------------------------------------------

                            print(f">>> Finished processing for prompt file: {prompt_file} <<<")

                    print(f"\n--- All prompts processed. Summaries saved to '{summaries_output_filepath}' ---")

                except IOError as e:
                     print(f"\nERROR: Could not write to summary file '{summaries_output_filepath}': {e}")
                     # Optional: Display results to console as a fallback if file write fails
                     print("\n--- Displaying Results to Console (File Write Failed) ---")
                     for prompt_f, results_list in all_results_by_prompt.items():
                          print(f"\n--- Results for Prompt: {prompt_f} ---")
                          if results_list:
                              for i, result in enumerate(results_list):
                                  print(f"Extraction {i+1}:\n{result}\n{'-'*20}")
                          else:
                              print(f"No relevant information extracted.")
                          print("="*30)


            else:
                print("Text file was read, but no chunks were created.") # Console
        else:
            print("Text file is empty.") # Console

    except FileNotFoundError:
        print(f"ERROR: Input text file not found at '{input_txt_filepath}'.") # Console
    except Exception as e:
        print(f"An unexpected error occurred during processing: {e}") # Console
        traceback.print_exc() # Prints traceback to console (stderr by default)

else:
    print("\nERROR: The 'pipe' object and/or 'tokenizer' object not found or not ready.") # Console
    print("Please ensure the Model Loading cell was run successfully BEFORE this cell.")

print(f"\n--- Processing Finished ---") # Console

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


--- Processing Started ---
Input file: /content/buyers-ID-manual-project/buyers-manual-article-01.txt
Attempting to save summaries to: extracted_summaries.txt
Pipeline and tokenizer are ready.
Reading text file: /content/buyers-ID-manual-project/buyers-manual-article-01.txt
Successfully read 65405 characters.
Chunking text with max_chunk_tokens = 2500...
DEBUG: nltk.data.find('tokenizers/punkt') succeeded inside chunk_text_by_tokens.
DEBUG: nltk.sent_tokenize produced 339 potential sentences.
Split text into 6 chunks using NLTK sentences (max tokens: 2500).
Opened 'extracted_summaries.txt' for writing summaries.

Processing prompts: ['prompt_main_idea.txt']

>>> Starting processing for prompt file: prompt_main_idea.txt <<<

Processing 6 chunks using prompt 'prompt_main_idea.txt' with batch_size=1...
  Formatting 6 inputs...
  Running pipeline on 6 inputs...
  Pipeline finished. Received 6 results.
  Filtering results...

Finished processing batch. Found 6 successful relevant extraction